<a href="https://colab.research.google.com/github/viridis45/Python-data-analysis/blob/master/20200111_NLP_stopwords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mecab 설치 (3~4분 소요)

In [0]:
import time
a = time.time()
#=============================================================
#-------------------------기본 설정----------------------------
#=============================================================

!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev

!pip3 install JPype1-py3 

!pip3 install konlpy

!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"



#=============================================================
#----------------------Mecab 다운로드--------------------------
#=============================================================
import os
os.chdir('/tmp/')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.1.tar.gz
!tar zxfv mecab-0.996-ko-0.9.1.tar.gz
os.chdir('/tmp/mecab-0.996-ko-0.9.1')
!./configure
!make
!make check
!make install

#=============================================================
#-----------------------automke 설치--------------------------
#=============================================================
! sudo apt-get update
! sudo apt-get install autoconf

os.chdir('/tmp')
!curl -LO http://ftpmirror.gnu.org/automake/automake-1.11.tar.gz
!tar -zxvf automake-1.11.tar.gz
os.chdir('/tmp/automake-1.11')
!./configure
!make
!make install

#=============================================================
#----------------------mecab-ko-dic  -------------------------
#=============================================================
import os
os.chdir('/tmp')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.0.1-20150920.tar.gz
!tar -zxvf mecab-ko-dic-2.0.1-20150920.tar.gz
os.chdir('/tmp/mecab-ko-dic-2.0.1-20150920')
!./autogen.sh
!./configure
!make

!make install



#=============================================================
#----------------------파이썬과 연동  -------------------------
#=============================================================
import os
os.chdir('/content')

!git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git
os.chdir('/content/mecab-python-0.996')

!python3 setup.py build
!python3 setup.py install

print(time.time()-a)

# Meacb 테스트

In [0]:
from konlpy.tag import Mecab
import pandas as pd
mecab = Mecab()

tagsets = pd.DataFrame()
N = 45
tagsets["Mecab-기호"] = list(mecab.tagset.keys()) + list("*" * (N - len(mecab.tagset))) 
tagsets["Mecab-품사"] = list(mecab.tagset.values()) + list("*" * (N - len(mecab.tagset)))

tagsets

In [0]:
from konlpy.tag import Mecab


mecab = Mecab()

print(mecab.morphs('안녕하세요'))
print(mecab.pos('안녕하세요'))
print(mecab.pos('햇살론')) # 햇살/ 론으로 구분되어 나와요
print(mecab.pos('CES'))
print(mecab.pos('QPR'))
print(mecab.pos('OSEN'))

['안녕', '하', '세요']
[('안녕', 'NNG'), ('하', 'XSV'), ('세요', 'EP+EF')]
[('햇살', 'NNG'), ('론', 'XSN')]
[('CES', 'SL')]
[('QPR', 'SL')]
[('OSEN', 'SL')]


In [0]:
b=dict()
for i in mecab.pos('안녕하세요'):
  
  for j in i[1].split('+'):
    if j in b:
      b[j]+=1
    else :
      b[j] = 1
b
    
  

{'EF': 1, 'EP': 1, 'NNG': 1, 'XSV': 1}

]# 구버젼 사전추가

In [0]:
import pandas as pd
nnp = pd.read_csv('/tmp/mecab-ko-dic-2.0.1-20150920/user-dic/nnp.csv')
nnp2 = nnp.copy()

# 새 단어 리스트
newwords=['햇살론'] #길이가 길어지면 csv파일을 불러오도록 수정합니다
for i in range(len(newwords)):
  nnp2.iloc[i,0], nnp2.iloc[i,7] = newwords[i], newwords[i]

# 사용자 단어파일 생성
nnp2.to_csv('/tmp/mecab-ko-dic-2.0.1-20150920/user-dic/nnp2.csv', index=False, header=False)


# 사용자 사전 추가
import os
os.chdir('/tmp/mecab-ko-dic-2.0.1-20150920/')
! tools/add-userdic.sh

os.chdir('/tmp/mecab-ko-dic-2.0.1-20150920/')
! make install

In [0]:
# 확인하기
from konlpy.tag import Mecab
mecab = Mecab()

print(mecab.morphs('안녕하세요'))
print(mecab.pos('안녕하세요'))
print(mecab.pos('햇살론')) # 햇살론이 추가됨 야호

['안녕', '하', '세요']
[('안녕', 'NNG'), ('하', 'XSV'), ('세요', 'EP+EF')]
[('햇살론', 'NNP')]


# 데이터 로드

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 키워드포함하는 숫자 치환하기

## 연습장

In [0]:
#===============================================
#             plan 2020.01.02-1225
#===============================================
#[숫자|X|-] 혼합 4개 이상
# '[\d|X]+-[\d|X|-]+' 숫자로만 있는것들은 어떻게 거를까요?
# : x를 하나도 포함하지 않는다면
# 키워드 거르기
#[\d|X|-]{5,}
#(entry[combination.start()-10:combination.end()])


#================데이터 불러오기
import pandas as pd
import regex as re
train = '/content/drive/My Drive/financo/data/train.csv'
trainfile = pd.read_csv(train).copy()
targetdata = trainfile[['text', 'smishing']].rename(columns={'text':'document', 'smishing':'label'})



#================사용할 데이터 구간 선택하기
sampledata = targetdata.copy() #전체
#sampledata = targetdata.iloc[:int(targetdata.shape[0]/3),:] # 전체의 1/3
#sampledata = targetdata.iloc[30:33,:].copy()


#================main
keywords = ['계좌','금액','통장','이체','예금주']

print(sampledata.shape)
altered = 0
combo = 0

lis = []
for entry in sampledata['document']:
#  for combination in re.finditer(r'[\d|X]+-[\d|X|-]+', entry):
  for combination in re.finditer(r'(\d|X)+-(\d|X|-)+', entry):
    if (combination.end()-combination.start()) > 4:
      combo+=1
      pad = str(entry)[combination.start()-15:combination.end()+15]
      lis.append(pad)
      for keyword in keywords:
        if keyword in pad:
          altered+=1
          break # 키워드가 발견되면 다음 combination으로 넘어가도록 합니다

print(f'총 {combo}개의 번호들 중 {altered}개가 해당됨. {altered/combo*100}%')


# 치환하기
# 다른 키워드들 고르기
# 나머지 개수


(295945, 2)
총 78355개의 번호들 중 495개가 해당됨. 0.6317401569778571%


In [0]:
import pandas as pd
import regex as re



def changeword(entry): #apply this onto column
  keywords = ['계좌','금액','통장','이체','예금주']
  phone = 'CES'
  bankaccount = 'QPR'
  for combination in re.finditer(r'(\d|X)+-(\d|X|-)+', entry):
    if len(combination.group()) > 4:
      for keyword in keywords:
        pad = entry[combination.start()-10 : combination.end()+10]
        if keyword in pad:
          combinationlen = combination.end() - combination.start()
          entry = entry[:combination.start()] + bankaccount + entry[combination.start()+combinationlen:]
  return entry


train = '/content/drive/My Drive/financo/data/train.csv'
trainfile = pd.read_csv(train).copy()
trainfile['text'] = trainfile['text'].apply(changeword)
trainfile['text'] = trainfile['text'].str.replace(r'(\d|X)+-(\d|X|-)+', 'CES')

trainfile[trainfile['text'].str.contains('CES')].shape




(42592, 5)

In [0]:
exportfilename = 'trainfile_200106.csv'

import os
os.chdir('/content/')
trainfile.to_csv( exportfilename, encoding='cp949', index=False)
! dir

drive  mecab-python-0.996  sample_data	trainfile_200106.csv


In [0]:
csvfile = '/content/mecab-python-0.996/processed_0111.csv'

tt = pd.read_csv(csvfile, encoding='cp949')

from konlpy.tag import Mecab
import pandas as pd
mecab = Mecab()

def Mecaber(raw_sentence):
  return ' '.join(mecab.morphs(raw_sentence))
import time

a = time.time()
print('processing...')
tt['text'] = tt['text'].apply(Mecaber)

print(time.time() - a)



processing...
118.69765114784241


In [0]:
! pwd

/content/mecab-python-0.996


In [0]:
tt.to_csv('train_p.csv', encoding='cp949')

In [0]:
mecabfilename = 'train200106_num_mec.csv'
tt.to_csv(mecabfilename, encoding='cp949', index=False)

In [0]:
ttt = pd.read_csv(mecabfilename, encoding='cp949')
ttt['text']=ttt['text'].str.replace('XXX', 'OSEN')
ttt.to_csv('num_mec_noun.csv', encoding='cp949', index=False)
ttt = pd.read_csv('num_mec_noun.csv', encoding='cp949')
ttt.head(4)

,Unnamed: 0,id,year_month,text,smishing
0,0,0,2017-01,OSEN 은 행성 산 OSEN 팀장 입니다 . 행복 한 주말 되 세요,0
1,1,1,2017-01,오늘 도 많이 웃 으시 는 하루 시작 하 세요 OSEN 은행 진월동 VIP 라운지 ...,0
2,2,2,2017-01,안녕 하 십니까 고객 님 . OSEN 은행 입니다 . 금일 납부 하 셔야 할 금액 ...,0
3,3,4,2017-01,OSEN 고객 님 안녕 하 세요 OSEN 은행 OSEN 지점 입니다 지난 한 해 동...,0


## one ring to rule them all

In [0]:
import pandas as pd
import re


def nums_mec_noun(csvfile, outputfilename):
  csvfile = pd.read_csv(csvfile)

  def changeword(entry): #apply this onto column
    keywords = ['계좌','금액','통장','이체','예금주']
    bankaccount = ' QPR '
    for combination in re.finditer(r'(\d|X)+-(\d|X|-)+', entry):
      if len(combination.group()) > 4:
        for keyword in keywords:
          pad = entry[combination.start()-10 : combination.end()+10]
          if keyword in pad:
            combinationlen = combination.end() - combination.start()
            entry = entry[:combination.start()] + bankaccount + entry[combination.start()+combinationlen:]
    return entry

  # mecab = Mecab()
  # from konlpy.tag import Mecab
  # def Mecaber(raw_sentence):
  #   return ' '.join(mecab.morphs(raw_sentence))


  csvfile['text'] = csvfile['text'].apply(changeword)
  csvfile['text'] = csvfile['text'].str.replace(r'(\d|X)+-(\d|X|-)+', ' CES ') #전화번호
  csvfile['text'] = csvfile['text'].str.replace('X{4,}', ' 국정원 ') # 기타번호
  csvfile['text'] = csvfile['text'].str.replace('X{3}', ' OSEN ') # 명사
  #csvfile['text'] = csvfile['text'].apply(Mecaber)

  csvfile.to_csv(outputfilename, encoding='cp949', index=False)


#nums_mec_noun('/content/drive/My Drive/financo/data/public_test.csv', 'test_01082020.csv')
#nums_mec_noun('/content/drive/My Drive/financo/data/train.csv', 'train_010920.csv')
# train.py에서 embed(), preprocessor.vocab.to_tokens(preprocessor.preprocess('안녕')))
# sklearn.metrics import roc_auc_score

## and in the darkness bind them_master

In [0]:
def XXX_btw_words(entry):
  pattern = re.compile('x', re.IGNORECASE)
  if len(pattern.findall(entry)) > (len(entry)*.75):
    entry=''
  return entry

def email_url_filter(text):
  e_pattern = r'''(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])'''
  u_pattern = r'http[s]?(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),|]|(?:%[0-9a-fA-F][0-9a-fA-F]))+' #당근마켓, 우리데이터에 맞게 변형
  u_pattern = r'[a-zA-Z1-9]*[.][a-zA-Z1-9.]*(co|com|kr|org)+'
  
  text = re.sub(e_pattern, ' 슈퍼스타 ', text)
  text = re.sub(u_pattern, ' 드래곤 ', text)
  return text


def bankaccount_filter(entry):
  keywords = ['계좌','금액','통장','이체','예금주']
  bankaccount = ' QPR '
  for combination in re.finditer(r'(\d|X|x)+-(\d|X|x|-)+', entry):
    if len(combination.group()) > 4:
      for keyword in keywords:
        pad = entry[combination.start()-10 : combination.end()+10]
        if keyword in pad:
          combinationlen = combination.end() - combination.start()
          entry = entry[:combination.start()] + bankaccount + entry[combination.start()+combinationlen:]
  return entry


def price_filter(text): 
  re_pattern = r'\d{1,3}[,\.]\d{1,3}[만\천]?\s?[원]|\d{1,5}[만\천]?\s?[원]'#당근마켓
  re_pattern2 = r'[일/이/삼/사/오/육/칠/팔/구/십/백][만\천]\s?[원]'#당근마켓
  re_pattern3 = r'[0-9]+( ){0,1}[원|만|억|천]+'
  
  text = re.sub(re_pattern, ' 황금어장 ', text)
  text = re.sub(re_pattern2, ' 황금어장 ', text)
  text = re.sub(re_pattern3, ' 황금어장 ', text)
  return text


def XXX_filter(text):
  telnum = r'(\d|X|x)+-(\d|X|x|-)+' # -를 포함하는 번호
  telnum2 = r'(\d{3}(-|.)\d{3}(-|.)\d{4})'
  othernum = r'\d{0,}[x|X]{4,}' # 3개 초과인 다른것들
  noun = r'[X|x]{3}' # XXX는 보통 고유명사
  noun2 = r'\W[X|x]{2}\W'
  
  text = re.sub(telnum, ' CES ', text)
  text = re.sub(telnum2, ' CES ', text)
  text = re.sub(othernum, ' 국정원 ', text)
  text = re.sub(noun, ' OSEN ', text)
  text = re.sub(noun2, ' OSEN ', text)

  return text

def numerical_tel_filter(text):
  tel = r'(\d{3}(-| |.)\d{3}(-|.| )\d{4})'
  text = re.sub(tel, ' CES ', text)
  return text




#==============================================
#
#                 main ish
#
#==============================================
import pandas as pd
import re
def master(csvfile, outputfilename):
  if type(csvfile) == str:
    csvfile = pd.read_csv(csvfile)

  csvfile['text'] = csvfile['text'].apply(XXX_btw_words)

  csvfile['text'] = csvfile['text'].apply(email_url_filter)
  csvfile['text'] = csvfile['text'].apply(bankaccount_filter)
  csvfile['text'] = csvfile['text'].apply(price_filter)
  csvfile['text'] = csvfile['text'].apply(XXX_filter)
  csvfile['text'] = csvfile['text'].apply(numerical_tel_filter)
  csvfile = csvfile[csvfile['text'].str.len() > 1]


  if type(outputfilename) == str:
    csvfile.to_csv(outputfilename, encoding='cp949', index=False)
  
  return csvfile


In [0]:
# 학습파일 전환

dd = pd.read_csv('/content/drive/My Drive/financo/data/train.csv')

import time
a = time.time()
master(dd, 'train_processed.csv')
print(time.time() - a)

34.41531276702881


In [0]:
! pwd

/content/mecab-python-0.996


In [0]:
# 테스트파일 전환
dd = pd.read_csv('/content/drive/My Drive/financo/data/public_test.csv')

import time
a = time.time()
master(dd, 'test.csv')
print(time.time() - a)

0.45470595359802246


# 기타

In [0]:
original = pd.read_csv('/content/drive/My Drive/financo/data/train.csv')
changed = pd.read_csv('/content/processed_0110.csv', encoding='cp949')

count = 0
same = 0
for i in range(original.shape[0]):
  if original['text'][i] != changed['text'][i]:
   count += 1
  else :
    same += 1

print(count)
print(same)


292880
3065


In [0]:
num = 1338
print(original[ original['id'] == num]['text'])
# print(changed[ changed['id'] == num]['text'])

1216    XXX 고객님XXX님 담보대출완제금액은20170117일기준으로원금: 165283186약정이자: 508721조기상환수수료: 1162374연체이자: 1785총금액: 166XXXXXX입니다.XXX은행옥포XXX올림
Name: text, dtype: object


In [0]:
changed[ changed['id'] == num]

,Unnamed: 0,id,year_month,text,smishing
1216,1216,1338,2017-01,OSEN 고객님 OSEN 님 담보대출완제금액은20170117일기준으로원금: 165283186약정이자: 508721조기상환수수료: 1162374연체이자: 1785총금액: 국정원 입니다. OSEN 은행옥포 OSEN 올림,0


In [0]:
original[ original['text'].str.contains('open.kakao') ]['smishing'].unique()

array([1])

In [0]:
changed[ changed['text'].str.contains('open.kakao') ]['smishing'].unique()

array([], dtype=int64)

## 뒤집기


In [0]:
original = pd.read_csv('/content/drive/My Drive/financo/data/train.csv')


ccandidates = pd.DataFrame()
candidates = original[ original['text'].str.len() > 100][['text','smishing']]
candidate = master(candidates, _)
print('rev completed')
candidate['text'] = candidate['text'].str[::-1]

# 뒤집어진 키워드 도로 뒤집기
rkw = []
kw = ['슈퍼스타','드래곤','국정원','QPR','CES','OSEN','황금어장']
for i in kw:
  rkw.append(i[::-1])

for i in range(len(rkw)):
  candidate['text'] = candidate['text'].str.replace(rkw[i], kw[i])



master(original, _).append(candidate, sort=False).to_csv('train.csv', encoding='cp949')

## 긴것들을 더해볼까

In [0]:
import pandas as pd
original = pd.read_csv('/content/mecab-python-0.996/train_p.csv', encoding='cp949')

In [0]:
original[original['text'].str.len() > 350]['smishing'].value_counts()

#스미싱 데이터가 총 18703 건이라는것을 감안하면 거의 모든 스미싱 데이터의 대부분이 읽히고있지 않는다는것을 알 수 있다 흑흑흑
# 지금은 250 토큰을 받아들이고 있기 떄문에 문장을 잘라서 넣어주는 방법을 사용?
# 어디서 잘라주는가가 관건이네
# 버트끼리 엮어서 끝에 히든레이어
# 앞, 끝부분을 잘라이어
# 클래시피케이션 레이어 증가시키기
# fully connected layer증가 -- 기존에 학습한것을 잃어버릴 수 있음 -- decay조정

0    33871
1    18583
Name: smishing, dtype: int64

In [0]:
candidates = pd.DataFrame()
candidates = original[ (original['text'].str.len() > 350) & (original['smishing'] == 1)][['text','smishing']]
def secondhalf(entry):
  return entry[350:]

candidates['text'] = candidates['text'].apply(secondhalf)


original.append(candidates, sort=False).to_csv('train_XXX_khaiii_append.csv', encoding='cp949')


In [0]:
original[original['text'].str.len() > 0]['smishing'].value_counts()

0    277242
1     18703
Name: smishing, dtype: int64

In [0]:
pd.read_csv('/content/mecab-python-0.996/test_p.csv', encoding = 'cp949')

,Unnamed: 0,id,year_month,text
0,0,340000,2019-01,OSEN 고객 님 ! 안녕 하 세요 ? 새롭 게 시작 하 는 한 주 행복 가득 하 ...
1,1,340001,2019-01,긴급 안내 OSEN 은행 가락동 지점 - 헬리오 OSEN 기본 OSEN 대출 이자 ...
2,2,340002,2019-01,OSEN 고객 님 안녕 하 세요 올해 는 미세먼지 가 유난 인 거 같 습니다 . 엊...
3,3,340003,2019-01,OSEN 고객 님 찾아온 행운 을 잡 으셨 나요 ? 못 잡 으셨다면 이번 에 다시 ...
4,4,340004,2019-01,OSEN 고객 님 새해 복 많이 받 으세요 OSEN 은행 코스트코 퇴직 연금 담당자...
...,...,...,...,...
1621,1621,341621,2019-04,( 광고 ) OSEN 고객 님 안녕 하 십니까 ! ! 고객 님 의 소중 한 업무 처...
1622,1622,341622,2019-04,고객 님 항상 저희 지점 을 거래 해 주 셔서 감사 합니다 본점 서비스 설문 전화 ...
1623,1623,341623,2019-04,OSEN 고객 님 저희 OSEN 은행 OSEN 지점 을 이용 해 주 셔서 감사 드립...
1624,1624,341624,2019-04,( 광고 ) 알림 신청 하 면 매일 현금 황금어장 ! OSEN 부동산 리브 온 지금...


In [0]:
original[(original['text'].str.len() > 350) & (original['smishing']==1)]['text'].str.len()

9          917
23         399
31         941
40         937
50         905
          ... 
295927     911
295932    1034
295934     911
295936     474
295937     826
Name: text, Length: 18108, dtype: int64